## Before you start this tutorial, get familiar with what an MD simulation looks like. You can look at the trajectory in this folder using the following command:
### Note: When you put a "!" at the beginning of a line in a jupyter notebook, it runs as if it was a BASH command. 

In [ ]:
! vmd exampleProtein.pdb exampleTraj.dcd 

### Note that, in VMD, we have to specify two files -- One containing the "topology" of the system (in this case a PDB file), which contains (at least) a listing of what atoms are present, what elements those atoms are, and how they are bonded. After that, we provide a "trajectory" file, or a compressed file containing X,Y,Z information about where each atom is in each simulation snapshot.

#### In addition to the elements and bonds, a PDB file contains a set of coordinates. This information goes beyond what is strictly expected in a "topology". If you want to view a stationary protein, a PDB file will suffice. Later you will learn about protein files like PRMTOPs that contain atoms and bonds, but no coordinates.



In [ ]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

### We load the trajectory using a topology file (in this case a pdb) and a trajectory file (the dcd)

### By the way, how did I know that mdtraj has a function called "load()"? I knew because I googled its documentation. I usually have documentation webpages open while I use different packages in programming.
### MDTraj's documentation can be found here: http://mdtraj.org/1.9.0/ . You should keep it open as you work on this notebook.

In [ ]:
traj = md.load('exampleTraj.dcd', top='exampleProtein.pdb')

### And get the number of frames in our trajectory

In [ ]:
numFrames = len(traj)
print(numFrames)

### Let's look at the atomic coordinates in our trajectory

In [ ]:
print(traj.xyz)

### Oof. That's a lot of numbers. Just how many numbers is it?

In [ ]:
print(traj.xyz.shape)

### Ah. This isn't so bad. It's 59,882 atoms, each with 3 coordinates (x, y, and z), and 51 simulation snapshots
### As an example, let's do something silly and move all the waters 100 Angstroms in the x direction.
### First we need to figure out which atoms are in waters.

In [ ]:
water_indices  = traj.topology.select('water')
print(waters.shape)

### This seems about right. 53,998 of our 59,882 atoms are waters.
### Remember numpy? It turns out that traj.xyz is a numpy array. So, to translate all the water molecules 100 angstroms (10 nanometers), we can just write:
#### (Note: Above we saw the shape is (51,59882,3). This means that the first index is the frame number, the second is the atom number, and the third is the spatial coordinate.)
#### (Also note: Jeff talks about distances in Angstroms, but MDTraj uses nanometers. 10 Angstroms = 1 nm)

In [ ]:
traj.xyz[:,water_indices,0] += 10

### I have no idea if this is right at all. Let's save out the first frame (traj[0]) as a pdb.

In [ ]:
traj[0].save_pdb('waters_translated_100.pdb')

In [ ]:
! vmd waters_translated_100.pdb

### How easy was that!?
## Ok. So we can do simple stuff with the water. Let's go further.
###  We know that waters play a very important role in protein function, because when proteins are put in environments other than water (ethanol, benzene, air, water that's too salty, or even just *heavy water*), they behave differently. Waters in the "bulk solvent", far away from the protein, generally just float ("diffuse") around, occasionally hydrogen bonding with each other, but never for long (unless it's cold. Then you get ice!). Waters around proteins, however, can hydrogen bond and get trapped for a long time. Sometimes these waters are actually necessary for the protein to function ("structural waters"). Frequently, binding sites will contain structural waters, and drug molecules can hydrogen bond to them and become more potent.
### One fairly advanced analysis that we can perform is a "water residence" analysis. This asks which waters stick around the protein for a while. For now, I just want to count how many waters stay near the protein between consecutive snapshots.
### First, we'll reload the protein, since we messed up that last structure by moving the waters around:

In [ ]:
traj = md.load('exampleTraj.dcd', top='exampleProtein.pdb')

### Second, we'll prepare a list to contain the atom numbers of each first shell water's oxygen in each snapshot

In [ ]:
firstShells = []

protein_indices = traj.topology.select('protein')
#waterSelection = traj.topology.select('water and name O')

### Then we'll use an MDTraj function to run through the trajectory and identify all the water oxygens within 2 angstroms of the protein. (Read up on this function I used: http://mdtraj.org/1.9.0/api/generated/mdtraj.compute_neighbors.html)

In [ ]:
watersNearProtein = md.compute_neighbors(traj, 
                                         0.2, 
                                         protein_indices,
                                         haystack_indices=water_indices)

### For each frame that was analyzed, put the water oxygen IDs into a set. I'll show you why sets are useful in a second.

In [ ]:
watersNearProtein = [set(i) for i in watersNearProtein]

### Walk through the waters near the protein in each frame (called the "first hydration shell"), and see how many of them are close in other frames. Make a (numFrames x numFrames) matrix with the number of first-shell waters that different frames have in common
### We will use the "&" (intersection) operator on two sets, which returns a new set containing the items that both sets have in common. Just like how two integers can be acted on with the "+", "-", "\*", and "/" operators, sets have their own special series of operators.

In [ ]:
firstOverlapMat = np.zeros((numFrames, numFrames))
for i in range(numFrames):
    for j in range(i, numFrames):
        overlap = len(watersNearProtein[i] & watersNearProtein[j])
        firstOverlapMat[i,j] = overlap
        firstOverlapMat[j,i] = overlap

### Use PyPlot to show what the first shell overlap matrix looks like

In [ ]:
plt.imshow(firstOverlapMat)
plt.colorbar()
plt.show()

### What does this mean? What are the yellow points and the purple? Why is there a diagonal yellow line?

In [ ]:
## Write down your explanation here

### Prepare a numpy array to hold the frame-to-frame similarity values

In [ ]:
firstOverlapMat = np.zeros((numFrames, numFrames))
nLeftList = []
nStayedList = []

### Now we go through each frame and compare it to the one right after it. We will use sets to do a handy task: Subtraction of two sets gives us a new set containing the items that are in one set and not another (difference). By counting how many items are in the difference and intersection, between every frame and the frame after it, we can count how many waters stay and leave each step.

In [ ]:
for frameIndex in range(numFrames-1):

    watersThatLeft = watersNearProtein[frameIndex] - watersNearProtein[frameIndex+1]
    # And the len() of a set is the number of items in it
    numWatersThatLeft = len(watersThatLeft)
    # The & operator returns items which appear in both 
    watersThatStayed = watersNearProtein[frameIndex] & watersNearProtein[frameIndex+1]
    numWatersThatStayed = len(watersThatStayed)
    
    print '%i waters left the first shell this frame, and %i stayed' %(numWatersThatLeft,numWatersThatStayed)

    # Store the results in a list
    nLeftList.append(numWatersThatLeft)
    nStayedList.append(numWatersThatStayed)

### And now we can use the PyPlot module to plot them

In [ ]:
plt.plot(nLeftList,label='Number of waters that left')
plt.plot(nStayedList,label='Number of waters that stayed')
plt.legend()
plt.show()


### There is an ATP molecule bound to this protein. Which protein residues does it contact? For how long? Write code to print out their residue name and the % of frames that they contact the ATP

In [ ]:
#Codecodecode